In [38]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
%matplotlib inline 

In [51]:
# link for dataset = 'https://code.datasciencedojo.com/datasciencedojo/datasets/tree/master/Fertility'
df = pd.read_csv('/content/fertility_Diagnosis.txt')
df.head()

,-0.33,0.69,0,1,1.1,0.1,0.8,0.2,0.88,N
0,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
1,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
2,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
3,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O
4,-0.33,0.67,1,0,1,0,0.8,0,0.50,N


Datasetimizni ustunlar va qatorlar sonini aniqlab olamiz

In [23]:
df.shape

(99, 10)

Datasetimizni ustunlari nomi mavjud emas shuning uchun uni ozimiz manual nomlaymiz

In [4]:
df.columns = ['Season', 'Age', 'Childish Diseases', 'Accidents', 'intervention', 'fevers', 'alcohol con', 'Smoking Habit', 'sit', 'label']
df.head()

,Season,Age,Childish Diseases,Accidents,intervention,fevers,alcohol con,Smoking Habit,sit,label
0,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
1,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
2,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
3,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O
4,-0.33,0.67,1,0,1,0,0.8,0,0.50,N


Datasetimizda nan qiymatlar mavjudmi yoki yoq tekshiramiz


In [5]:
df.isna().sum()

Season               0
Age                  0
Childish Diseases    0
Accidents            0
intervention         0
fevers               0
alcohol con          0
Smoking Habit        0
sit                  0
label                0
dtype: int64

Bir hil malumotlar(duplicate) bormi yoki yoqligini tekshiramiz

In [6]:
df.duplicated().sum()

0

Correlyatsiya mavjudligini tekshirib korish uchun yangi DataFrame yaratib olamiz va undagi label ni Binary sonlariga o'zgartirib olamiz va correlyatsiyani tekshiramiz

In [13]:
df2 = df.copy()
df2.replace('O', 0, inplace = True)
df2.replace('N', 1, inplace = True)
df2.head()

,Season,Age,Childish Diseases,Accidents,intervention,fevers,alcohol con,Smoking Habit,sit,label
0,-0.33,0.94,1,0,1,0,0.8,1,0.31,0
1,-0.33,0.50,1,0,0,0,1.0,-1,0.50,1
2,-0.33,0.75,0,1,1,0,1.0,-1,0.38,1
3,-0.33,0.67,1,1,0,0,0.8,-1,0.50,0
4,-0.33,0.67,1,0,1,0,0.8,0,0.50,1


Korrelyatsiya deyarli mavjud emas shuning uchun bu datasetda regressiya algoritmlaridan foydalanish unchalik qo'l kelmaydi

In [14]:
df2.corr()

,Season,Age,Childish Diseases,Accidents,intervention,fevers,alcohol con,Smoking Habit,sit,label
Season,1.000000,0.066010,-0.191464,-0.093313,-0.003091,-0.223105,-0.041934,-0.026732,-0.011239,-0.191464
Age,0.066010,1.000000,0.088141,0.215397,0.271585,0.120947,-0.247687,0.071897,-0.462468,-0.115976
Childish Diseases,-0.191464,0.088141,1.000000,0.200562,-0.120052,0.069479,0.034721,0.105631,-0.086882,0.051724
Accidents,-0.093313,0.215397,0.200562,1.000000,0.093046,-0.079084,-0.242140,0.105985,-0.016613,0.138123
intervention,-0.003091,0.271585,-0.120052,0.093046,1.000000,-0.229584,-0.074332,-0.058093,-0.226632,-0.058150
fevers,-0.223105,0.120947,0.069479,-0.079084,-0.229584,1.000000,-0.001469,-0.006092,-0.147632,0.122799
alcohol con,-0.041934,-0.247687,0.034721,-0.242140,-0.074332,-0.001469,1.000000,-0.184292,0.120366,0.145604
Smoking Habit,-0.026732,0.071897,0.105631,0.105985,-0.058093,-0.006092,-0.184292,1.000000,-0.121388,-0.047592
sit,-0.011239,-0.462468,-0.086882,-0.016613,-0.226632,-0.147632,0.120366,-0.121388,1.000000,-0.033628
label,-0.191464,-0.115976,0.051724,0.138123,-0.058150,0.122799,0.145604,-0.047592,-0.033628,1.000000


Datasetimizni X va Y ga bo'lib olamiz yani X bu bashorat qilishda foydalaniladigon malumotlar, Y esa bashorat qilinuvchi malumot

In [21]:
X = df.drop(columns = 'label')
Y = df['label']

train_test_split funksiyasi yordamida datasetimizni ikkiga bo'lib olamiz

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state = 4, test_size = 0.1)

LogisticRegression model da train qilganimizda 90 foiz aniqlik bilan ishlamoqda prediction, endi K-NN algoritmidan ham foydalanib koramiz


In [36]:
model = LogisticRegression() 
model.fit(x_train, y_train)
prediction = model.predict(x_test)
accuracy_score(prediction,y_test)


0.9

Datasetimiz kichik bo'lgani uchun 

In [50]:
for i in range(1,6):
  model2 = KNeighborsClassifier(n_neighbors=i)
  model2.fit(x_train, y_train)
  prediction2 = model2.predict(x_test)
  print('_____________________________________',i)
  print(classification_report(prediction2, y_test))

_____________________________________ 1
              precision    recall  f1-score   support

           N       0.67      1.00      0.80         6
           O       1.00      0.25      0.40         4

    accuracy                           0.70        10
   macro avg       0.83      0.62      0.60        10
weighted avg       0.80      0.70      0.64        10

_____________________________________ 2
              precision    recall  f1-score   support

           N       1.00      0.90      0.95        10
           O       0.00      0.00      0.00         0

    accuracy                           0.90        10
   macro avg       0.50      0.45      0.47        10
weighted avg       1.00      0.90      0.95        10

_____________________________________ 3
              precision    recall  f1-score   support

           N       1.00      0.90      0.95        10
           O       0.00      0.00      0.00         0

    accuracy                           0.90        10
   macro

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMe

Dataset kichikligi sabab deyarli barcha modellar birhil ishlamoqda, eng yuqori natija 90% ligicha qoldi
